## MODULE:1 FACE DETECTION

In [ ]:
# TOOLS OR LIBRARIES REQUIRED
# MEDIAPIPE FACE DETECTION


In [18]:
import cv2
import math
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# =========================
# MODEL INITIALIZATION
# =========================

base_face_det = python.BaseOptions(model_asset_path=r"E:\Projects in ML\FRAUD DETECTION SYSTEM FOR THE ONLINE PROCTORED EXAMS\mediapipe_models\blaze_face_short_range.tflite")
base_face_lm  = python.BaseOptions(model_asset_path=r"E:\Projects in ML\FRAUD DETECTION SYSTEM FOR THE ONLINE PROCTORED EXAMS\mediapipe_models\face_landmarker.task")
base_hand     = python.BaseOptions(model_asset_path=r"E:\Projects in ML\FRAUD DETECTION SYSTEM FOR THE ONLINE PROCTORED EXAMS\mediapipe_models\hand_landmarker.task")

face_detector = vision.FaceDetector.create_from_options(
    vision.FaceDetectorOptions(
        base_options=base_face_det,
        running_mode=vision.RunningMode.IMAGE
    )
)

face_landmarker = vision.FaceLandmarker.create_from_options(
    vision.FaceLandmarkerOptions(
        base_options=base_face_lm,
        output_facial_transformation_matrixes=True,
        running_mode=vision.RunningMode.IMAGE
    )
)

hand_landmarker = vision.HandLandmarker.create_from_options(
    vision.HandLandmarkerOptions(
        base_options=base_hand,
        num_hands=2,
        running_mode=vision.RunningMode.IMAGE
    )
)

# =========================
# UTILITY FUNCTIONS
# =========================

def euclidean(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def compute_head_pose(lm, w, h):
    try:
        required_ids = [1, 33, 61, 152, 263, 291]
        for idx in required_ids:
            if idx >= len(lm):
                return None, None, None

        model_points = np.array([
            (0, 0, 0),
            (0, -330, -65),
            (-225, 170, -135),
            (225, 170, -135),
            (-150, -150, -125),
            (150, -150, -125)
        ], dtype=np.float64)

        image_points = np.array([
            (lm[1].x * w, lm[1].y * h),
            (lm[152].x * w, lm[152].y * h),
            (lm[33].x * w, lm[33].y * h),
            (lm[263].x * w, lm[263].y * h),
            (lm[61].x * w, lm[61].y * h),
            (lm[291].x * w, lm[291].y * h)
        ], dtype=np.float64)

        cam = np.array([
            [w, 0, w / 2],
            [0, w, h / 2],
            [0, 0, 1]
        ], dtype=np.float64)

        dist = np.zeros((4, 1))

        success, rvec, _ = cv2.solvePnP(
            model_points,
            image_points,
            cam,
            dist,
            flags=cv2.SOLVEPNP_ITERATIVE
        )

        if not success:
            return None, None, None

        rmat, _ = cv2.Rodrigues(rvec)
        pitch, yaw, roll = cv2.RQDecomp3x3(rmat)[0]
        return pitch, yaw, roll

    except Exception:
        return None, None, None

def head_pose_label(pitch, yaw):
    if abs(pitch) < 10 and abs(yaw) < 10:
        return "forward"
    if yaw > 10:
        return "right"
    if yaw < -10:
        return "left"
    if pitch > 10:
        return "down"
    return "None"

# =========================
# MAIN FEATURE EXTRACTION
# =========================

def extract_features(image_path):
    img = cv2.imread(image_path)
    h, w, _ = img.shape
    mp_img = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)

    features = {}

    # -------- FACE DETECTION --------
    face_res = face_detector.detect(mp_img)
    features["no_of_face"] = len(face_res.detections)
    features["face_present"] = 1 if face_res.detections else 0

    if not face_res.detections:
        return features

    det = face_res.detections[0]
    bbox = det.bounding_box

    features.update({
        "face_x": bbox.origin_x,
        "face_y": bbox.origin_y,
        "face_w": bbox.width,
        "face_h": bbox.height,
        "face_conf": det.categories[0].score * 100
    })

    # -------- FACE LANDMARKS --------
    mesh = face_landmarker.detect(mp_img)
    lm = mesh.face_landmarks[0]

    features.update({
        "left_eye_x": lm[33].x * w,
        "left_eye_y": lm[33].y * h,
        "right_eye_x": lm[263].x * w,
        "right_eye_y": lm[263].y * h,
        "nose_tip_x": lm[1].x * w,
        "nose_tip_y": lm[1].y * h,
        "mouth_x": lm[13].x * w,
        "mouth_y": lm[13].y * h
    })

    # -------- PUPILS & GAZE --------
    plx, ply = lm[468].x * w, lm[468].y * h
    prx, pry = lm[473].x * w, lm[473].y * h

    gaze_x = (plx + prx) / 2
    gaze_y = (ply + pry) / 2

    if gaze_x < 0.4 * w:
        gaze_dir = "bottom_left"
    elif gaze_x > 0.6 * w:
        gaze_dir = "bottom_right"
    else:
        gaze_dir = "center"

    features.update({
        "pupil_left_x": plx,
        "pupil_left_y": ply,
        "pupil_right_x": prx,
        "pupil_right_y": pry,
        "gazePoint_x": gaze_x,
        "gazePoint_y": gaze_y,
        "gaze_direction": gaze_dir,
        "gaze_on_script": 1 if gaze_dir == "center" else 0
    })

    # -------- HEAD POSE --------
    # pitch, yaw, roll = compute_head_pose(lm, w, h)
    # features.update({
    #     "head_pitch": pitch,
    #     "head_yaw": yaw,
    #     "head_roll": roll,
    #     "head_pose": head_pose_label(pitch, yaw)
    # })
    pitch, yaw, roll = compute_head_pose(lm, w, h)

    if pitch is None:
        features.update({
            "head_pitch": 0,
            "head_yaw": 0,
            "head_roll": 0,
            "head_pose": "None"
        })
    else:
        features.update({
            "head_pitch": pitch,
            "head_yaw": yaw,
            "head_roll": roll,
            "head_pose": head_pose_label(pitch, yaw)
        })


    # -------- HANDS --------
    hands = hand_landmarker.detect(mp_img)
    features["hand_count"] = len(hands.hand_landmarks)

    features.update({
        "left_hand_x": 0,
        "left_hand_y": 0,
        "right_hand_x": 0,
        "right_hand_y": 0,
        "hand_obj_interaction": 0
    })

    if hands.hand_landmarks:
        wrist = hands.hand_landmarks[0][0]
        features["left_hand_x"] = wrist.x
        features["left_hand_y"] = wrist.y

        mouth_pt = (features["mouth_x"], features["mouth_y"])
        wrist_px = (wrist.x * w, wrist.y * h)

        if euclidean(mouth_pt, wrist_px) < 80:
            features["hand_obj_interaction"] = 1

    return features

# =========================
# TEST RUN
# =========================




In [19]:
# ==============================
# RUN TEST
# ==============================
count=0
if __name__ == "__main__":
    feats = extract_features("input.jpg")
    for k, v in feats.items():
        print(f"{k}: {v}")
        count+=1
        print("Count:",count)

no_of_face: 1
Count: 1
face_present: 1
Count: 2
face_x: 473
Count: 3
face_y: 285
Count: 4
face_w: 305
Count: 5
face_h: 305
Count: 6
face_conf: 70.30482888221741
Count: 7
left_eye_x: 533.1682968139648
Count: 8
left_eye_y: 370.53189754486084
Count: 9
right_eye_x: 705.1795196533203
Count: 10
right_eye_y: 360.4400110244751
Count: 11
nose_tip_x: 636.0640335083008
Count: 12
nose_tip_y: 439.9025774002075
Count: 13
mouth_x: 636.6813659667969
Count: 14
mouth_y: 497.76546478271484
Count: 15
pupil_left_x: 558.5663604736328
Count: 16
pupil_left_y: 369.1128158569336
Count: 17
pupil_right_x: 681.1087799072266
Count: 18
pupil_right_y: 362.22928047180176
Count: 19
gazePoint_x: 619.8375701904297
Count: 20
gazePoint_y: 365.6710481643677
Count: 21
gaze_direction: center
Count: 22
gaze_on_script: 1
Count: 23
head_pitch: -179.58198741217254
Count: 24
head_yaw: -14.79146484952375
Count: 25
head_roll: -3.8087009262764004
Count: 26
head_pose: left
Count: 27
hand_count: 0
Count: 28
left_hand_x: 0
Count: 29
lef